In [8]:
library(devtools)

#get the current file path without an RStudio API dependency
current_path = "C:/Users/mgcal/OneDrive/Documents/School/Research/Genetics/cTWAS Generalization/Code/test_susine"
setwd(current_path)

load_all("..") #load the test_susine package

ERROR: [1m[33mError[39m in `value[[3L]]()`:[22m
[1m[22m[33m![39m Could not find a root 'DESCRIPTION' file that starts with '^Package'
  in [34mC:\Users\mgcal\OneDrive\Documents\School\Research\Genetics\cTWAS[39m
  [34mGeneralization\Code[39m.
[36mℹ[39m Are you in your project directory and does your project have a 'DESCRIPTION'
  file?


### Generate batch controls

In [ ]:
experiment_name = "SuSiE X, simulated annotations, and simulated phenotype"

L = c(1,3,5,10,20) #number of SuSiNE model effects
y_noise = c(0.5, 0.75, 0.85, 0.90, 0.95) #proportion of variance not explained by genetic effects within locus
p_star = c(1,5,10,20) #number of true causal effects
annotation_noise = c(0, 0.2, 0.4, 0.6, 0.8, 1)
n_seeds = 10

batch_controls = expand.grid(experiment_name = experiment_name,
                             L = L,
                             y_noise = y_noise,
                             p_star = p_star,
                             seed = 1:n_seeds)

#Add the annotation noise column as list (each row has multiple values)
# make a list-column: same vector in every row
batch_controls$annotation_noise <- rep(list(annotation_noise), nrow(batch_controls))

head(batch_controls)

#Create output directory if it doesn't exist
dir.create(file.path("..", "output", "temp"), recursive = TRUE, showWarnings = FALSE)

#Save batch_controls to the output/temp folder for review
save(batch_controls, file = file.path("..", "output","temp", sprintf("%s_batch_controls.RData", gsub("[^A-Za-z0-9._-]+","_", experiment_name))))

#Generate slurm batch file (shell script), save to the output/temp folder for review
generate_batch_file(experiment_name, nrow(batch_controls))

### Kick off the batch

In [ ]:
data("SuSiE_N3_X") #This comes from the SuSiE pkg under 'data/'N3finemapping.rData'

#Get n, p from X data
X = as.matrix(SuSiE_N3_X)
X = standardize_X(X)
rm(SuSiE_N3_X)
p = ncol(X)
n =  nrow(X)